In [ ]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl')
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu
import dl.loaders.ultrasound_dataset as ultrasound_dataset

import importlib

from dl.nets.layers import TimeDistributed

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src')
from shapeaxi import utils as saxi_utils
from shapeaxi.saxi_transforms import EvalTransform

from pytorch3d.ops import (sample_points_from_meshes,
                           knn_points, 
                           knn_gather,
                           sample_farthest_points)


In [ ]:
mount_point = '/mnt/raid/C1_ML_Analysis'

importlib.reload(us_simu)
vs = us_simu.VolumeSamplingBlindSweep(mount_point=mount_point)
vs = vs.cuda()

In [ ]:
batch_size = 1
dm = ultrasound_dataset.ImgPCDataModule(
    mount_point="/mnt/raid/C1_ML_Analysis/simulated_data_export",
    csv_train="simulated_data_export/studies_fetus_train_train.csv",
    np_train="simulated_data_export/studies_fetus_train_train.npy",
    csv_valid="simulated_data_export/studies_fetus_train_test.csv",
    np_valid="simulated_data_export/studies_fetus_train_test.npy",
    csv_test="simulated_data_export/studies_fetus_test.csv",
    np_test="simulated_data_export/studies_fetus_test.npy",
    num_samples_train=5000,
    rescale_factor=1,
    batch_size=batch_size,
    num_workers=1
    )
dm.setup()

it = iter(dm.test_dataloader())

In [ ]:
X, X_origin, X_end, X_PC = next(it)

In [ ]:
NN = getattr(us_simu, "USSeg")
model = NN.load_from_checkpoint("train_output/seg/USSeg/0.1/last.ckpt")

In [ ]:
x, y, sweeps_tags = model.volume_sampling(X.cuda(), X_origin.cuda(), X_end.cuda())
x = x.squeeze(1)

# x_hat = self(x)

In [ ]:
fig = px.imshow(x.squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
fig.show()